In [111]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torchtext import data
import spacy
import os
from torch.autograd import Variable

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(USE_CUDA)

False


# Hyperparameters

In [112]:
batch_size_train = 5
batch_size_eval = 256
embed_dim = 50
embed_from = "glove.6B.50d"
hidden_size = 96
num_layers = 1
rnn_dropout_rate = 0
embed_dropout_rate = 0.4

# grad_clipping!!!!

# Load data
refer to 

http://anie.me/On-Torchtext/

http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/

http://mlexplained.com/2018/02/15/language-modeling-tutorial-in-torchtext-practical-torchtext-part-2/

In [113]:
data_dir = 'preprocessed'
combined_fname = 'all-combined-data-processed.json'
train_fname = 'train-trial-combined-data-processed.json'
dev_fname = 'dev-data-processed.json'
test_fname = 'test-data-processed.json'

we have keys: 'id', 'd_words', 'd_pos', 'd_ner', 'q_words', 'q_pos', 'c_words', 'label', 'in_q', 'in_c', 'lemma_in_q', 'tf', 'p_q_relation', 'p_c_relation'

In [114]:
TEXT = data.ReversibleField(sequential=True, lower=True, include_lengths=True)

train, val, test = data.TabularDataset.splits(
    path=data_dir, train=train_fname,
    validation=dev_fname, test=test_fname, format='json',
    fields={'d_words': ('d_words', TEXT),
            'q_words': ('q_words', TEXT),
            'c_words': ('c_words', TEXT),
            'label': ('label', data.Field(sequential=False, use_vocab=False))
           })

print('train: %d, val: %d, test: %d' % (len(train), len(val), len(test)))

train: 20482, val: 2822, test: 5594


In [115]:
# combined is only used for building vocabulary
combined = data.TabularDataset(
    path=os.path.join(data_dir, combined_fname), format='json',
    fields={'d_words': ('d_words', TEXT),
            'q_words': ('q_words', TEXT),
            'c_words': ('c_words', TEXT),
            'label': ('label', data.Field(sequential=False, use_vocab=False))
           })

# TEXT.build_vocab(combined)
TEXT.build_vocab(combined, vectors=embed_from)
print('vocab size: %d' % len(TEXT.vocab))

vocab size: 10695


In [116]:
train_iter, val_iter, test_iter = data.Iterator.splits(
        (train, val, test), batch_sizes=(batch_size_train, batch_size_eval, batch_size_eval), device=device, 
        sort_within_batch=False, repeat=False)

# Create embedding

In [117]:
embedding = nn.Embedding(len(TEXT.vocab), embed_dim)
embedding.weight.data.copy_(TEXT.vocab.vectors)
embedding.weight.requires_grad=False
embedding = embedding.to(device)

In [118]:
embedding.weight.shape

torch.Size([10695, 50])

# Build model
refer to

https://github.com/intfloat/commonsense-rc

https://towardsdatascience.com/taming-lstms-variable-sized-mini-batches-and-why-pytorch-is-good-for-your-health-61d35642972e

https://discuss.pytorch.org/t/solved-multiple-packedsequence-input-ordering/2106/23

In [212]:
class BLSTM(nn.Module):
    def __init__(self, input_dim, hidden_size, num_layers, rnn_dropout_rate):
        super(BLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=rnn_dropout_rate,
            bidirectional=True
        )
    
    def forward(self, inputs, lengths):
        """
        take inputs (embedded and padded), return outputs from lstm
        
        :param inputs: (batch_size, seq_len, embed_dim)
        :param lengths: (batch_size)
        :return: (batch_size, seq_len, hidden_size * 2)
        """
        lengths_sorted, sorted_idx = lengths.sort(descending=True)
        inputs_sorted = inputs[sorted_idx]
    
        inputs_packed = nn.utils.rnn.pack_padded_sequence(inputs_sorted, lengths_sorted.tolist(), batch_first=True)
        outputs_packed, _ = self.lstm(inputs_packed)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs_packed, batch_first=True)
        
        # Reverses sorting. 
        outputs = torch.zeros_like(outputs)\
            .scatter_(0, sorted_idx.unsqueeze(1).unsqueeze(1)
                      .expand(-1, outputs.shape[1], outputs.shape[2]), outputs)
        
        return outputs

In [213]:
def lengths_to_mask(lengths, dtype=torch.uint8):
    """
    
    :param lengths: (batch_size)
    :param dtype: 
    :return: (batch_size, max_len)
    """
    max_len = lengths.max().item()
    mask = torch.arange(max_len,
                        dtype=lengths.dtype).expand(len(lengths), max_len) < lengths.unsqueeze(1)

    mask = torch.as_tensor(mask, dtype=dtype, device=device)
    mask = 1 - mask
    return mask

In [256]:
class SeqAttnContext(nn.Module):
    def __init__(self, embed_dim):
        super(SeqAttnContext, self).__init__()
        
        self.proj = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU()
        )
        
        self.softmax = nn.Softmax(dim=2)

    def forward(self, x, y, y_mask):
        """
        calculate context vectors for x on y using attention on y
        
        :param x: (batch_size, x_seq_len, embed_dim)
        :param y: (batch_size, y_seq_len, embed_dim)
        :param y_lengths: (batch_size)
        :return: (batch_size, x_seq_len, embed_dim)
        """
        x_proj = self.proj(x)
        y_proj = self.proj(y)
        
        scores = x_proj.bmm(y_proj.transpose(2, 1))
        
        # mask scores
        y_mask = y_mask.unsqueeze(1).expand(scores.size())
        
        scores.data.masked_fill_(y_mask.data, -float('inf'))
        weights = self.softmax(scores)
        
        # Take weighted average
        contexts = weights.bmm(y)
        # here, instead of using y, maybe use another projection of y in the future
        
        return contexts

In [257]:
class SelfAttnEncoder(nn.Module):
    def __init__(self, input_dim):
        super(SelfAttnEncoder, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, inputs, mask):
        """
        Summarize inputs into single vectors using self attention
        
        :param self: 
        :param inputs: (batch_size, seq_len, input_dim)
        :param mask: (batch_size, seq_len)
        :return: (batch_size, input_dim)
        """
        scores = self.linear(inputs).squeeze(2)
        scores.data.masked_fill_(mask.data, -float('inf'))
        weights = self.softmax(scores) # (batch_size, seq_len)
        
        return weights.unsqueeze(1).bmm(inputs).squeeze(1)

In [258]:
class TriAn(nn.Module):
    def __init__(self, embedding):
        super(TriAn, self).__init__()
        self.embedding = embedding
        self.d_rnn = BLSTM(embed_dim * 2, hidden_size, num_layers, rnn_dropout_rate)
        self.q_rnn = BLSTM(embed_dim, hidden_size, num_layers, rnn_dropout_rate)
        self.c_rnn = BLSTM(embed_dim * 3, hidden_size, num_layers, rnn_dropout_rate)
        
        self.embed_dropout = nn.Dropout(embed_dropout_rate)
        
        self.d_on_q_attn = SeqAttnContext(embed_dim)
        self.c_on_q_attn = SeqAttnContext(embed_dim)
        self.c_on_d_attn = SeqAttnContext(embed_dim)
        
        
    
    def forward(self, d_words, d_lengths, q_words, q_lengths, c_words, c_lengths):
        # embed inputs
        d_embed, q_embed, c_embed = self.embedding(d_words), \
            self.embedding(q_words), self.embedding(c_words)
        d_embed, q_embed, c_embed = self.embed_dropout(d_embed), self.embed_dropout(q_embed),\
            self.embed_dropout(c_embed)
        
        # get masks
        d_mask = lengths_to_mask(d_lengths)
        q_mask = lengths_to_mask(q_lengths)
        c_mask = lengths_to_mask(c_lengths)
        
        # get attention contexts
        d_on_q_contexts = self.embed_dropout(self.d_on_q_attn(d_embed, q_embed, q_mask))
        c_on_q_contexts = self.embed_dropout(self.c_on_q_attn(c_embed, q_embed, q_mask))
        c_on_d_contexts = self.embed_dropout(self.c_on_d_attn(c_embed, d_embed, d_mask))
        
        # form final inputs for rnns
        d_rnn_inputs = torch.cat([d_embed, d_on_q_contexts], dim=2)
        q_rnn_inputs = torch.cat([q_embed], dim=2)
        c_rnn_inputs = torch.cat([c_embed, c_on_q_contexts, c_on_d_contexts], dim=2)
        
        # calculate rnn outputs
        d_rnn_outputs = self.d_rnn(d_rnn_inputs, d_lengths)
        q_rnn_outputs = self.q_rnn(q_rnn_inputs, q_lengths)
        c_rnn_outputs = self.c_rnn(c_rnn_inputs, c_lengths)
        
        # get final representations
        d_rep = 
        
        
        
        
        
        
        return None
        

In [259]:
model = TriAn(embedding).to(device)

In [260]:
for i, batch in enumerate(train_iter):
    d_words, d_lengths = batch.d_words
    q_words, q_lengths = batch.q_words
    c_words, c_lengths = batch.c_words
    
    d_words, d_lengths = torch.transpose(d_words, 0, 1).to(device), d_lengths.to(device)
    q_words, q_lengths = torch.transpose(q_words, 0, 1).to(device), q_lengths.to(device)
    c_words, c_lengths = torch.transpose(c_words, 0, 1).to(device), c_lengths.to(device)
    
    labels = batch.label.to(device)
    
    outputs = model(d_words, d_lengths, q_words, q_lengths, c_words, c_lengths)


#     # for testing
#     c_embed = embedding(c_words)
#     c_mask = lengths_to_mask(c_lengths)
# #     q_embed = embedding(q_words)

#     sae = SelfAttnEncoder(embed_dim)
#     outputs = sae(c_embed, c_mask)
#     print(c_embed.shape)
#     print(outputs.shape)
    
    

    if i == 0:
        break

torch.Size([5, 6, 50])
torch.Size([5, 50])
